In [1]:
# 获取指定目录下的所有文件
from nmc_met_io.retrieve_micaps_server import get_file_list

file_list = get_file_list(
    "ECMWF_HR/TMP/850",
    latest=10,
)
file_list

['20031420.240',
 '20031420.234',
 '20031420.228',
 '20031420.222',
 '20031420.216',
 '20031420.210',
 '20031420.204',
 '20031420.198',
 '20031420.192',
 '20031420.186']

In [2]:
# 返回最新的起报时间
from nmc_met_io.retrieve_micaps_server import get_latest_initTime
get_latest_initTime("ECMWF_HR/TMP/850")

'20031420'

In [3]:
# 读取单个时次模式标量预报数据
from nmc_met_io.retrieve_micaps_server import get_model_grid
directory = 'ECMWF_HR/TMP/850'
filename = '20031108.024'
data = get_model_grid(
    directory, 
    filename=filename, 
    cache=False
)
if data is not None:
    print(data)
else:
    print("Retrieve failed.")
    

<xarray.Dataset>
Dimensions:                  (lat: 281, level: 1, lon: 361, time: 1)
Coordinates:
  * time                     (time) datetime64[ns] 2020-03-12T08:00:00
  * level                    (level) float32 850.0
  * lat                      (lat) float64 60.0 59.75 59.5 ... -9.5 -9.75 -10.0
  * lon                      (lon) float64 60.0 60.25 60.5 ... 149.5 149.8 150.0
    forecast_reference_time  datetime64[ns] 2020-03-11T08:00:00
    forecast_period          (time) float64 24.0
Data variables:
    data                     (time, level, lat, lon) float32 -2.6221583 ... 19.830967
Attributes:
    Conventions:  CF-1.6
    Origin:       MICAPS Cassandra Server


In [4]:
# 可以指定数据的变量名称, 变量属性等信息
data = get_model_grid(
    directory, 
    filename=filename, 
    varname='TEM', 
    varattrs={'long_name':'temperature', 'units':'degree'}, 
    cache=False
)
if data is not None:
    print(data.TEM)
else:
    print("Retrieve failed.")

<xarray.DataArray 'TEM' (time: 1, level: 1, lat: 281, lon: 361)>
array([[[[ -2.6221583,  -2.3409083,  -1.9659082, ..., -12.387783 ,
          -13.622158 , -14.684658 ],
         [ -1.2940332,  -1.3096582,  -1.0909082, ..., -11.856533 ,
          -12.106533 , -12.856533 ],
         [ -0.1065332,  -0.3409082,  -0.2471582, ..., -14.372158 ,
          -14.122158 , -13.981533 ],
         ...,
         [ 18.424717 ,  18.409092 ,  18.502842 , ...,  19.159092 ,
           19.252842 ,  19.159092 ],
         [ 18.471592 ,  18.393467 ,  18.565342 , ...,  19.377842 ,
           19.330967 ,  19.362217 ],
         [ 18.502842 ,  18.455967 ,  18.502842 , ...,  20.112217 ,
           19.815342 ,  19.830967 ]]]], dtype=float32)
Coordinates:
  * time                     (time) datetime64[ns] 2020-03-12T08:00:00
  * level                    (level) float32 850.0
  * lat                      (lat) float64 60.0 59.75 59.5 ... -9.5 -9.75 -10.0
  * lon                      (lon) float64 60.0 60.25 60.5 ... 1

In [5]:
# 获取多个数据
from nmc_met_io.retrieve_micaps_server import get_model_grids
dataset = get_model_grids(
    directory, 
    filenames=[f"20031108.{i:03}" for i in range(0, 25, 3)], 
    pbar=True,
    cache=False
)
dataset

ECMWF_HR/TMP/850: 100%|██████████| 9/9 [00:00<00:00, 14.73it/s]


<xarray.Dataset>
Dimensions:                  (lat: 281, level: 1, lon: 361, time: 9)
Coordinates:
    forecast_reference_time  datetime64[ns] 2020-03-11T08:00:00
  * lon                      (lon) float64 60.0 60.25 60.5 ... 149.5 149.8 150.0
  * lat                      (lat) float64 60.0 59.75 59.5 ... -9.5 -9.75 -10.0
  * level                    (level) float32 850.0
  * time                     (time) datetime64[ns] 2020-03-11T08:00:00 ... 2020-03-12T08:00:00
    forecast_period          (time) float64 0.0 3.0 6.0 9.0 ... 18.0 21.0 24.0
Data variables:
    data                     (time, level, lat, lon) float32 -3.9067652 ... 19.830967
Attributes:
    Conventions:  CF-1.6
    Origin:       MICAPS Cassandra Server

In [22]:
import numpy as np

from nmc_met_io.retrieve_micaps_server import get_model_points
directory = "NWFD_SCMOC/TMP/2M_ABOVE_GROUND"
forecast_hours = np.arange(3, 75, 3)
filenames = [f"20031408.{hour:03}" for hour in forecast_hours]
points = {
    'lon':[116.3833, 110.0], 
    'lat':[39.9, 32],
}
data = get_model_points(directory, filenames, points)
points = data["data"]
points

<xarray.DataArray 'data' (time: 24, level: 1, points: 2)>
array([[[13.69967175, 11.26998594]],

       [[14.62996202, 15.23000765]],

       [[15.12631122, 16.63997965]],

       [[11.43631271, 10.1399771 ]],

       [[ 8.04640026,  4.27998505]],

       [[ 6.26624725,  3.3999855 ]],

       [[ 4.93257821,  2.60998089]],

       [[ 6.5230763 ,  4.69997664]],

       [[13.8299143 , 13.82000623]],

       [[15.23339242, 19.70998893]],

       [[15.47323917, 19.92998769]],

       [[11.50967601, 13.65999475]],

       [[ 7.2464106 ,  7.83998504]],

       [[ 4.72661674,  6.18998897]],

       [[ 1.97330037,  5.49999387]],

       [[ 2.64697281,  8.04000368]],

       [[ 4.77351423, 12.01000686]],

       [[12.09575915, 14.71999614]],

       [[12.59013931, 15.23998473]],

       [[ 8.69924855, 11.99999397]],

       [[ 6.19290629,  9.0599925 ]],

       [[ 5.52294008,  7.23998621]],

       [[ 4.668902  ,  6.77999308]],

       [[ 7.32416252,  7.4800055 ]]])
Coordinates:
    forecast_reference_time  datetime64[ns] 2020-03-14T08:00:00
  * level                    (level) float32 2.0
  * time                     (time) datetime64[ns] 2020-03-14T11:00:00 ... 2020-03-17T08:00:00
    forecast_period          (time) float64 3.0 6.0 9.0 12.0 ... 66.0 69.0 72.0
    lon                      (points) float64 116.4 110.0
    lat                      (points) float64 39.9 32.0
Dimensions without coordinates: points
Attributes:
    units:

In [23]:
points.sel(points=0)

<xarray.DataArray 'data' (time: 24, level: 1)>
array([[13.69967175],
       [14.62996202],
       [15.12631122],
       [11.43631271],
       [ 8.04640026],
       [ 6.26624725],
       [ 4.93257821],
       [ 6.5230763 ],
       [13.8299143 ],
       [15.23339242],
       [15.47323917],
       [11.50967601],
       [ 7.2464106 ],
       [ 4.72661674],
       [ 1.97330037],
       [ 2.64697281],
       [ 4.77351423],
       [12.09575915],
       [12.59013931],
       [ 8.69924855],
       [ 6.19290629],
       [ 5.52294008],
       [ 4.668902  ],
       [ 7.32416252]])
Coordinates:
    forecast_reference_time  datetime64[ns] 2020-03-14T08:00:00
  * level                    (level) float32 2.0
  * time                     (time) datetime64[ns] 2020-03-14T11:00:00 ... 2020-03-17T08:00:00
    forecast_period          (time) float64 3.0 6.0 9.0 12.0 ... 66.0 69.0 72.0
    lon                      float64 116.4
    lat                      float64 39.9
Attributes:
    units: